In [10]:
import numpy as np
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import sqlite3
import os

### 1. Get the data from our existing SQlite DB

In [2]:
nba_sqlite = sqlite3.connect('nba.db')

In [3]:
players = pd.read_sql('SELECT * FROM players', nba_sqlite)
player_game = pd.read_sql('SELECT * FROM player_game', nba_sqlite)
team_game = pd.read_sql('SELECT * FROM team_game', nba_sqlite)
games = pd.read_sql('SELECT * FROM games', nba_sqlite)

### 2. Connect to Postgres Server

In [5]:
pgpassword = os.environ['POSTGRES_PASSWORD']

In [6]:
server = psycopg2.connect(
    user = 'postgres',
    password = pgpassword,
    host = 'postgres',
    port = '5432'
)
server.autocommit = True

### 3. Create a new empty database for the NBA data

In [8]:
cursor = server.cursor()

In [9]:
try: 
    cursor.execute("CREATE DATABASE nba")
except:
    cursor.execute("DROP DATABASE nba")
    cursor.execute("CREATE DATABASE nba")

### 4. Load the four NBA tables into this Postgres DB

In [11]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = pgpassword,
    host = 'postgres',
    port = '5432',
    db = 'nba'))

In [16]:
games.to_sql('games', con = engine, if_exists = 'replace', chunksize = 1000, index = False)
players.to_sql('players', con = engine, if_exists = 'replace', chunksize = 1000, index = False)
team_game.to_sql('team_game', con = engine, if_exists = 'replace', chunksize = 1000, index = False)
player_game.to_sql('player_game', con = engine, if_exists = 'replace', chunksize = 1000, index = False)

31608

### 5. Do some SQL queries

In [17]:
myquery = '''
SELECT * 
FROM games
'''
pd.read_sql(myquery, con=engine)

,game_id,game_date,OT
0,202204100BRK,2022-04-10,0
1,202204100CHO,2022-04-10,0
2,202204100CLE,2022-04-10,0
3,202204100DAL,2022-04-10,0
4,202204100DEN,2022-04-10,1
...,...,...,...
1225,202110300MIN,2021-10-30,0
1226,202112150DEN,2021-12-15,0
1227,202202010MIN,2022-02-01,0
1228,202203270BOS,2022-03-27,0
